# Deceleration Analysis for Traffic Light Placement

## Objective:
Based on our EDA findings, we know this is high-quality highway data with free-flowing traffic.
Now we need to identify where vehicles naturally decelerate to place realistic traffic lights.

## Approach:
1. Calculate deceleration from speed changes
2. Identify significant deceleration events  
3. Analyze spatial clustering of events
4. Recommend traffic light placement locations

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Force matplotlib to use inline backend
%matplotlib inline

# Add the parent directory to sys.path (useful to resolve module import errors)
sys.path.append(str(Path().resolve().parent.parent))

# Load the Data saved from EDA
tracks_df = pd.read_csv('../data/processed/highd_cleaned.csv')  # Save from EDA


## Analyzing Stopping Patterns

In [ ]:
# Find slow-moving vehicles
SLOW_SPEED_THRESHOLD = 22  # m/s (about 79 km/h)
slow_events = tracks_df[tracks_df['speed'] < SLOW_SPEED_THRESHOLD]

display(tracks_df[["speed"]].describe())

#### Calculate_deceleration:
Function to calculate the deceleration value for each vehicle. It sorts the tracks by vehicle id and frame. 
Example:

From
| id | frame | speed |
|----|-------|-------|
| 1  | 5     | 30    |
| 2  | 1     | 25    |
| 1  | 3     | 28    |
| 1  | 4     | 29    |
| 2  | 2     | 26    |

To

| id | frame | speed |
|----|-------|-------|
| 1  | 3     | 28    |
| 1  | 4     | 29    |
| 1  | 5     | 30    |
| 2  | 1     | 25    |
| 2  | 2     | 26    |

It creates a speed column if it does not exist, and later groups the data by id and uses `.diff` to calculate the difference between consecutive rows.
The deceleration is calculated by the formula: `acceleration = change in speed / Time elapsed`. 
- Data is recorded at 25 Hz (a.k.a: FRAME_RATE = 25 frames per second)
- Each frame = 1/25 = 0.04 seconds apart
- If speed changes by -2 m/s in one frame

Deceleration = -2 m/s ÷ 0.04 seconds = -50 m/s² = (-2*25)

In [ ]:
def calculate_deceleration(df):
    """
    Calculate the deceleration for each vehicle trajectory
    
    @params:
        - df: tracks dataframe
    @returns:
        - df_sorted: Sorted df with two new columns: 'speed_change' and 'deceleration'
    """
    
    # Sort by vehicle ID and frame to ensure proper time ordering
    df_sorted = tracks_df.sort_values(['id', 'frame']).copy()
    
    if 'speed' not in df_sorted.columns:
        df_sorted['speed'] = np.sqrt(df_sorted['xVelocity']**2 + df_sorted['yVelocity']**2)
    
    df_sorted['speed_change'] = df_sorted.groupby('id')['speed'].diff()
    
    df_sorted['deceleration'] = df_sorted['speed_change'] * 25
    
    print(f"Calculated deceleration for {df_sorted['id'].nunique()} vehicles")
    
    return df_sorted

In [ ]:
tracks_with_decel = calculate_deceleration(tracks_df)

display(tracks_with_decel[["deceleration", "speed_change"]].describe())

### Identify Significant deceleration events

In [ ]:
deceleration_threshold = -2.0  # m/s² (negative = slowing down)

# Find significant deceleration events
decel_events = tracks_with_decel[tracks_with_decel['deceleration'] < deceleration_threshold].copy()

print(f"Deceleration threshold: {deceleration_threshold} m/s²")
print(f"Found {len(decel_events):,} significant deceleration events")
print(f"Percentage of data: {len(decel_events)/len(tracks_with_decel)*100:.2f}%")
print(f"Number of vehicles with decel events: {decel_events['id'].nunique()}")

if len(decel_events) > 0:
    print(f"\nDeceleration event statistics:\n" + "-"*6)
    print(f"Average deceleration: {decel_events['deceleration'].mean():.2f} m/s²")
    print(f"Strongest deceleration: {decel_events['deceleration'].min():.2f} m/s²")
    
    # Show sample events
    print(f"\nSample deceleration events:\n" + "-"*6)
    sample_events = decel_events[['id', 'frame', 'x', 'y', 'speed', 'deceleration', 'laneId']].head(10)
    display(sample_events.head())

##### Plot 1: Deceleration distribution

In [ ]:
plt.figure(figsize=(14, 4))

plt.subplot(1, 2, 1)
plt.hist(tracks_with_decel['deceleration'], bins=100, alpha=0.7, density=True, color='skyblue', edgecolor='black')
plt.axvline(deceleration_threshold, color='red', linestyle='--', linewidth=2, 
            label=f'Threshold: {deceleration_threshold} m/s²')
plt.axvline(0, color='green', linestyle='-', linewidth=1, label='No change')
plt.xlabel('Deceleration (m/s²)')
plt.ylabel('Frequency')
plt.title('Distribution of Deceleration Events')
plt.legend()
plt.grid(True, alpha=0.3)

##### Plot 2: Acceleration vs Deceleration

In [ ]:
plt.subplot(1, 2, 2)
acceleration_events = tracks_with_decel[tracks_with_decel['deceleration'] > 2.0]
plt.hist(tracks_with_decel['deceleration'], bins=50, alpha=0.5, density=True, label='All events', color='lightblue')
plt.hist(decel_events['deceleration'], bins=30, alpha=0.7, density=True, label='Significant decel', color='red')
plt.hist(acceleration_events['deceleration'], bins=30, alpha=0.7, density=True, label='Significant accel', color='green')
plt.xlabel('Deceleration/Acceleration (m/s²)')
plt.ylabel('Frequency')
plt.title('Deceleration vs Acceleration Events')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
## Spatial Analysis - Where Do Deceleration Events Happen?

In [ ]:
plt.figure(figsize=(15, 6))
    
# Background: all vehicle positions (light intensity)
plt.scatter(tracks_with_decel['x'], tracks_with_decel['y'], 
           c='lightblue', alpha=0.1, s=1, label='All positions')

# Foreground: deceleration events (colored by intensity)
scatter = plt.scatter(decel_events['x'], decel_events['y'], 
                     c=decel_events['deceleration'], cmap='Reds_r', 
                     alpha=0.7, s=20, 
                     label='Deceleration events')

plt.colorbar(scatter, label='Deceleration (m/s²)')
plt.xlabel('X Position (meters)')
plt.ylabel('Y Position (meters)')
plt.title('Spatial Distribution of Deceleration Events\n(Darker red = stronger deceleration)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()